In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [7]:
from sklearn.feature_selection import mutual_info_classif

mi_scores= mutual_info_classif(X_sm_tl, y_sm_tl)

In [8]:
feature_names = X.columns
mi_df= pd.DataFrame({'Feature': feature_names,
                     'Mutual Information': mi_scores})

mi_df= mi_df.sort_values(by='Mutual Information', ascending=False)
mi_df.head(10)

,Feature,Mutual Information
16,BMI,0.176943
9,Exercise Hours Per Week,0.175458
21,LBP,0.172441
3,Heart Rate,0.168381
20,HBP,0.167474
0,Age,0.167016
13,Stress Level,0.154217
18,Physical Activity Days Per Week,0.150138
14,Sedentary Hours Per Day,0.149247
19,Sleep Hours Per Day,0.132206


In [9]:
k= 10
top_k_features= mi_df.head(k)['Feature'].values
top_k_indices= [feature_names.get_loc(f) for f in top_k_features]

X_sm_tl_selected= X_sm_tl[:, top_k_indices]
X_sm_tl_test_selected= X_test[:, top_k_indices]

--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.64      0.50      0.56      1125
           1       0.35      0.49      0.41       628

    accuracy                           0.50      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.50      0.51      1753

Precision: 0.3548387096774194
Accuracy: 0.4980034227039361
AUC: 0.48570842179759377


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.64      0.64      1125
           1       0.37      0.38      0.38       628

    accuracy                           0.55      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.55      0.55      1753

Precision: 0.3707692307692308
Accuracy: 0.5459212778094695
AUC: 0.5101012031139419


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 20}
Best Precision Score: 0.631999315095747


In [13]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.64      0.93      0.76      1125
           1       0.36      0.07      0.12       628

    accuracy                           0.62      1753
   macro avg       0.50      0.50      0.44      1753
weighted avg       0.54      0.62      0.53      1753

Precision: 0.36220472440944884
Accuracy: 0.6217912150598973
AUC: 0.5036036801132342


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.66      0.50      0.57      1125
           1       0.38      0.54      0.44       628

    accuracy                           0.51      1753
   macro avg       0.52      0.52      0.51      1753
weighted avg       0.56      0.51      0.52      1753

Precision: 0.37679558011049724
Accuracy: 0.5145464917284654
AUC: 0.5150976645435245


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.6987452054013431


In [16]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.75      0.69      1125
           1       0.35      0.24      0.28       628

    accuracy                           0.57      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.57      0.54      1753

Precision: 0.34872979214780603
Accuracy: 0.5670279520821448
AUC: 0.4915796178343949


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.64      0.51      0.57      1125
           1       0.36      0.49      0.42       628

    accuracy                           0.51      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.51      0.52      1753

Precision: 0.36065573770491804
Accuracy: 0.5059897318881916
AUC: 0.4992285916489738


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.65      0.86      0.74      1125
           1       0.39      0.16      0.23       628

    accuracy                           0.61      1753
   macro avg       0.52      0.51      0.48      1753
weighted avg       0.55      0.61      0.56      1753

Precision: 0.3886792452830189
Accuracy: 0.6081003993154592
AUC: 0.5085428167020523


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.65      0.88      0.75      1125
           1       0.42      0.16      0.24       628

    accuracy                           0.62      1753
   macro avg       0.54      0.52      0.49      1753
weighted avg       0.57      0.62      0.56      1753

Precision: 0.425
Accuracy: 0.6212207644038791
AUC: 0.5117070063694267


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.7802682219026023


In [22]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.65      0.89      0.75      1125
           1       0.42      0.14      0.21       628

    accuracy                           0.62      1753
   macro avg       0.54      0.52      0.48      1753
weighted avg       0.57      0.62      0.56      1753

Precision: 0.42452830188679247
Accuracy: 0.623502567027952
AUC: 0.5117105449398442


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1125
           1       0.37      0.37      0.37       628

    accuracy                           0.55      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.55      0.55      1753

Precision: 0.3745980707395498
Accuracy: 0.5527666856816885
AUC: 0.509343949044586


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6944587523490255


In [26]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.64      0.86      0.73      1125
           1       0.37      0.15      0.21       628

    accuracy                           0.60      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.60      0.55      1753

Precision: 0.3657587548638132
Accuracy: 0.6023958927552767
AUC: 0.49725831564048123


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.96      0.77      1125
           1       0.32      0.03      0.06       628

    accuracy                           0.63      1753
   macro avg       0.48      0.50      0.41      1753
weighted avg       0.53      0.63      0.51      1753

Precision: 0.3220338983050847
Accuracy: 0.6297775242441529
AUC: 0.5155513092710545


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7459884008007835


In [29]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.65      0.95      0.77      1125
           1       0.44      0.07      0.13       628

    accuracy                           0.63      1753
   macro avg       0.55      0.51      0.45      1753
weighted avg       0.57      0.63      0.54      1753

Precision: 0.44339622641509435
Accuracy: 0.6349115801483172
AUC: 0.5138202406227884


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.65      0.79      0.71      1125
           1       0.38      0.23      0.29       628

    accuracy                           0.59      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.59      0.56      1753

Precision: 0.37894736842105264
Accuracy: 0.5892755276668569
AUC: 0.51593347487615


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7060226390582929
              precision    recall  f1-score   support

           0       0.65      0.89      0.75      1125
           1       0.41      0.13      0.20       628

    accuracy                           0.62      1753
   macro avg       0.53      0.51      0.48      1753
weighted avg       0.56      0.62      0.56      1753

Precision: 0.41379310344827586
Accuracy: 0.6217912150598973
AUC: 0.518965322009908


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1125
           1       0.45      0.13      0.20       628

    accuracy                           0.63      1753
   macro avg       0.55      0.52      0.48      1753
weighted avg       0.58      0.63      0.56      1753

Precision: 0.4519774011299435
Accuracy: 0.6320593268682259
AUC: 0.5335612172682236


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000591 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.7627087902915802
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_le

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.02579
0:	learn: 0.6909272	total: 148ms	remaining: 2m 27s
1:	learn: 0.6855428	total: 153ms	remaining: 1m 16s
2:	learn: 0.6832243	total: 157ms	remaining: 52.3s
3:	learn: 0.6825112	total: 162ms	remaining: 40.3s
4:	learn: 0.6803699	total: 167ms	remaining: 33.2s
5:	learn: 0.6790138	total: 171ms	remaining: 28.4s
6:	learn: 0.6772858	total: 176ms	remaining: 25s
7:	learn: 0.6759349	total: 182ms	remaining: 22.5s
8:	learn: 0.6736032	total: 187ms	remaining: 20.6s
9:	learn: 0.6714072	total: 193ms	remaining: 19.1s
10:	learn: 0.6700750	total: 198ms	remaining: 17.8s
11:	learn: 0.6678313	total: 203ms	remaining: 16.7s
12:	learn: 0.6656288	total: 208ms	remaining: 15.8s
13:	learn: 0.6642707	total: 213ms	remaining: 15s
14:	learn: 0.6618922	total: 218ms	remaining: 14.3s
15:	learn: 0.6603296	total: 223ms	remaining: 13.7s
16:	learn: 0.6589012	total: 229ms	remaining: 13.2s
17:	learn: 0.6575564	total: 234ms	remaining: 12.7s
18:	learn: 0.6564193	total: 239ms	remaining: 12.3s
19:	learn: 0.6

180:	learn: 0.5409405	total: 1.05s	remaining: 4.75s
181:	learn: 0.5407328	total: 1.05s	remaining: 4.74s
182:	learn: 0.5405355	total: 1.06s	remaining: 4.73s
183:	learn: 0.5403159	total: 1.06s	remaining: 4.73s
184:	learn: 0.5401170	total: 1.07s	remaining: 4.72s
185:	learn: 0.5398892	total: 1.08s	remaining: 4.71s
186:	learn: 0.5388516	total: 1.08s	remaining: 4.7s
187:	learn: 0.5386148	total: 1.09s	remaining: 4.7s
188:	learn: 0.5383474	total: 1.09s	remaining: 4.69s
189:	learn: 0.5379179	total: 1.1s	remaining: 4.68s
190:	learn: 0.5373878	total: 1.1s	remaining: 4.68s
191:	learn: 0.5371862	total: 1.11s	remaining: 4.67s
192:	learn: 0.5368207	total: 1.11s	remaining: 4.66s
193:	learn: 0.5364165	total: 1.12s	remaining: 4.65s
194:	learn: 0.5360803	total: 1.13s	remaining: 4.65s
195:	learn: 0.5357964	total: 1.13s	remaining: 4.64s
196:	learn: 0.5355654	total: 1.14s	remaining: 4.63s
197:	learn: 0.5352701	total: 1.14s	remaining: 4.63s
198:	learn: 0.5349966	total: 1.15s	remaining: 4.62s
199:	learn: 0.53

350:	learn: 0.4968603	total: 1.94s	remaining: 3.59s
351:	learn: 0.4966702	total: 1.95s	remaining: 3.58s
352:	learn: 0.4965438	total: 1.95s	remaining: 3.58s
353:	learn: 0.4963933	total: 1.96s	remaining: 3.57s
354:	learn: 0.4961995	total: 1.96s	remaining: 3.57s
355:	learn: 0.4960384	total: 1.97s	remaining: 3.56s
356:	learn: 0.4958033	total: 1.98s	remaining: 3.56s
357:	learn: 0.4955775	total: 1.98s	remaining: 3.55s
358:	learn: 0.4952356	total: 1.99s	remaining: 3.55s
359:	learn: 0.4951025	total: 1.99s	remaining: 3.54s
360:	learn: 0.4949086	total: 2s	remaining: 3.54s
361:	learn: 0.4945103	total: 2s	remaining: 3.53s
362:	learn: 0.4942343	total: 2.01s	remaining: 3.52s
363:	learn: 0.4940343	total: 2.01s	remaining: 3.52s
364:	learn: 0.4938884	total: 2.02s	remaining: 3.51s
365:	learn: 0.4936758	total: 2.02s	remaining: 3.51s
366:	learn: 0.4935139	total: 2.03s	remaining: 3.5s
367:	learn: 0.4932672	total: 2.04s	remaining: 3.5s
368:	learn: 0.4930078	total: 2.04s	remaining: 3.49s
369:	learn: 0.492850

514:	learn: 0.4577720	total: 2.84s	remaining: 2.67s
515:	learn: 0.4575877	total: 2.84s	remaining: 2.67s
516:	learn: 0.4574191	total: 2.85s	remaining: 2.66s
517:	learn: 0.4572243	total: 2.85s	remaining: 2.66s
518:	learn: 0.4570716	total: 2.86s	remaining: 2.65s
519:	learn: 0.4567968	total: 2.87s	remaining: 2.65s
520:	learn: 0.4565768	total: 2.87s	remaining: 2.64s
521:	learn: 0.4563722	total: 2.88s	remaining: 2.63s
522:	learn: 0.4561583	total: 2.88s	remaining: 2.63s
523:	learn: 0.4559214	total: 2.89s	remaining: 2.62s
524:	learn: 0.4557617	total: 2.89s	remaining: 2.62s
525:	learn: 0.4555320	total: 2.9s	remaining: 2.61s
526:	learn: 0.4553511	total: 2.9s	remaining: 2.61s
527:	learn: 0.4551667	total: 2.91s	remaining: 2.6s
528:	learn: 0.4549444	total: 2.92s	remaining: 2.6s
529:	learn: 0.4547860	total: 2.92s	remaining: 2.59s
530:	learn: 0.4546615	total: 2.93s	remaining: 2.58s
531:	learn: 0.4544910	total: 2.93s	remaining: 2.58s
532:	learn: 0.4542666	total: 2.94s	remaining: 2.57s
533:	learn: 0.45

675:	learn: 0.4281022	total: 3.72s	remaining: 1.78s
676:	learn: 0.4279228	total: 3.73s	remaining: 1.78s
677:	learn: 0.4277282	total: 3.74s	remaining: 1.77s
678:	learn: 0.4275145	total: 3.74s	remaining: 1.77s
679:	learn: 0.4273510	total: 3.75s	remaining: 1.76s
680:	learn: 0.4271474	total: 3.75s	remaining: 1.76s
681:	learn: 0.4270210	total: 3.76s	remaining: 1.75s
682:	learn: 0.4268559	total: 3.77s	remaining: 1.75s
683:	learn: 0.4267042	total: 3.77s	remaining: 1.74s
684:	learn: 0.4264910	total: 3.78s	remaining: 1.74s
685:	learn: 0.4263037	total: 3.79s	remaining: 1.73s
686:	learn: 0.4262054	total: 3.79s	remaining: 1.73s
687:	learn: 0.4260118	total: 3.8s	remaining: 1.72s
688:	learn: 0.4258656	total: 3.8s	remaining: 1.72s
689:	learn: 0.4257105	total: 3.81s	remaining: 1.71s
690:	learn: 0.4255557	total: 3.81s	remaining: 1.71s
691:	learn: 0.4253693	total: 3.82s	remaining: 1.7s
692:	learn: 0.4251973	total: 3.83s	remaining: 1.69s
693:	learn: 0.4250547	total: 3.83s	remaining: 1.69s
694:	learn: 0.4

840:	learn: 0.4014372	total: 4.62s	remaining: 873ms
841:	learn: 0.4013030	total: 4.62s	remaining: 868ms
842:	learn: 0.4011331	total: 4.63s	remaining: 862ms
843:	learn: 0.4010217	total: 4.63s	remaining: 857ms
844:	learn: 0.4008353	total: 4.64s	remaining: 851ms
845:	learn: 0.4007125	total: 4.64s	remaining: 845ms
846:	learn: 0.4005693	total: 4.65s	remaining: 840ms
847:	learn: 0.4004635	total: 4.65s	remaining: 834ms
848:	learn: 0.4002916	total: 4.66s	remaining: 829ms
849:	learn: 0.4001391	total: 4.67s	remaining: 823ms
850:	learn: 0.4000213	total: 4.67s	remaining: 818ms
851:	learn: 0.3998911	total: 4.67s	remaining: 812ms
852:	learn: 0.3997148	total: 4.68s	remaining: 807ms
853:	learn: 0.3994896	total: 4.68s	remaining: 801ms
854:	learn: 0.3993186	total: 4.69s	remaining: 795ms
855:	learn: 0.3992084	total: 4.69s	remaining: 790ms
856:	learn: 0.3990186	total: 4.7s	remaining: 784ms
857:	learn: 0.3988877	total: 4.7s	remaining: 779ms
858:	learn: 0.3987512	total: 4.71s	remaining: 773ms
859:	learn: 0.

In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6896182	total: 4.11ms	remaining: 407ms
1:	learn: 0.6847445	total: 7.69ms	remaining: 377ms
2:	learn: 0.6822809	total: 11.1ms	remaining: 359ms
3:	learn: 0.6788740	total: 14.6ms	remaining: 350ms
4:	learn: 0.6760817	total: 18.4ms	remaining: 349ms
5:	learn: 0.6740425	total: 21.8ms	remaining: 341ms
6:	learn: 0.6716756	total: 25.9ms	remaining: 344ms
7:	learn: 0.6684661	total: 29.3ms	remaining: 337ms
8:	learn: 0.6660170	total: 33.2ms	remaining: 336ms
9:	learn: 0.6630058	total: 37.9ms	remaining: 341ms
10:	learn: 0.6611760	total: 42ms	remaining: 340ms
11:	learn: 0.6591175	total: 45.9ms	remaining: 337ms
12:	learn: 0.6565942	total: 49.6ms	remaining: 332ms
13:	learn: 0.6541096	total: 53.2ms	remaining: 327ms
14:	learn: 0.6519504	total: 56.5ms	remaining: 320ms
15:	learn: 0.6491100	total: 60ms	remaining: 315ms
16:	learn: 0.6474214	total: 63.7ms	remaining: 311ms
17:	learn: 0.6457138	total: 67.3ms	remaining: 306ms
18:	learn: 0.643

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
15                        LGBM Precision:         0.451977
17                    CatBoost Precision:         0.443787
12      Tuned GradientBoosting Precision:         0.443396
7                Random Forest Precision:         0.425000
8          Tuned Random Forest Precision:         0.424528
14                   Tuned XGB Precision:         0.413793
6                          SVM Precision:         0.388679
13                         XGB Precision:         0.378947
3         KNeighborsClassifier Precision:         0.376796
9                     AdaBoost Precision:         0.374598
18              Tuned CatBoost Precision:         0.373016
1                Decision Tree Precision:         0.370769
10              Tuned AdaBoost Precision:         0.365759
2          Tuned Decision Tree Precision:         0.362205
5                   GaussianNB Precision:         0.360656
0          Logistic Regression Precision:         0.3548

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
12      Tuned GradientBoosting Accuracy:        0.634912
15                        LGBM Accuracy:        0.632059
17                    CatBoost Accuracy:        0.630918
11            GradientBoosting Accuracy:        0.629778
16                  Tuned LGBM Accuracy:        0.628066
18              Tuned CatBoost Accuracy:        0.623503
8          Tuned Random Forest Accuracy:        0.623503
14                   Tuned XGB Accuracy:        0.621791
2          Tuned Decision Tree Accuracy:        0.621791
7                Random Forest Accuracy:        0.621221
6                          SVM Accuracy:        0.608100
10              Tuned AdaBoost Accuracy:        0.602396
13                         XGB Accuracy:        0.589276
4   Tuned KNeighborsClassifier Accuracy:        0.567028
9                     AdaBoost Accuracy:        0.552767
1                Decision Tree Accuracy:        0.545921
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
15                        LGBM AUC:   0.533561
14                   Tuned XGB AUC:   0.518965
13                         XGB AUC:   0.515933
11            GradientBoosting AUC:   0.515551
3         KNeighborsClassifier AUC:   0.515098
12      Tuned GradientBoosting AUC:   0.513820
18              Tuned CatBoost AUC:   0.513595
8          Tuned Random Forest AUC:   0.511711
7                Random Forest AUC:   0.511707
16                  Tuned LGBM AUC:   0.510309
1                Decision Tree AUC:   0.510101
9                     AdaBoost AUC:   0.509344
6                          SVM AUC:   0.508543
17                    CatBoost AUC:   0.506406
2          Tuned Decision Tree AUC:   0.503604
5                   GaussianNB AUC:   0.499229
10              Tuned AdaBoost AUC:   0.497258
4   Tuned KNeighborsClassifier AUC:   0.491580
0          Logistic Regression AUC:   0.485708
